In [1]:
from pymatgen import Structure, Lattice, PeriodicSite
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar, Incar
from pymatgen.analysis.elasticity.strain import Deformation
Si_DC = Structure.from_file("my_part_1/Si_DC/CONTCAR")
print(Si_DC)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.866890   3.866890   3.866890
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.5   0.5   0.5
  1  Si    0.75  0.75  0.75


In [2]:
POSCAR = Poscar(Si_DC)
print(POSCAR)

Si2
1.0
-0.000000 2.734304 2.734304
2.734304 0.000000 2.734304
2.734304 2.734304 -0.000000
Si
2
direct
0.500000 0.500000 0.500000 Si
0.750000 0.750000 0.750000 Si



In [3]:
my_incar_params = {"EDIFF" : 0.0004, 
                "EDIFFG" : -0.01, 
                "ENCUT" : 350, 
                "IBRION" : 2, 
                "ISIF" : 2, 
                "ISMEAR" : 0, 
                "SIGMA" : 0.2, 
                "NSW" : 15,
                "NBANDS" : 8,
                "LREAL" : False}

In [4]:
INCAR = Incar.from_dict(my_incar_params)
print(INCAR)

EDIFF = 0.0004
EDIFFG = -0.01
ENCUT = 350
IBRION = 2
ISIF = 2
ISMEAR = 0
LREAL = False
NBANDS = 8
NSW = 15
SIGMA = 0.2



In [5]:
# fill in this tuple with desired grid dimensions (e.g. (a, b, c))
grid = (8, 8, 8)
# We're using the monkhorst_automatic constructor here to create a grid via the Monkhorst-Pack method.
KPOINTS = Kpoints.monkhorst_automatic(kpts=grid)
print(KPOINTS)

Automatic kpoint scheme
0
Monkhorst
8 8 8



In [6]:
#Define our strain tensor. In this case, e_11 = e_22 = e_33 = 1.10.
strain_tensor = ((0.9, 0, 0), (0, 0.9, 0), (0, 0, 0.9))

deformation = Deformation(strain_tensor)

In [7]:
deformed_Si_DC = deformation.apply_to_structure(Si_DC)
print(deformed_Si_DC)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.480201   3.480201   3.480201
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.5   0.5   0.5
  1  Si    0.75  0.75  0.75


In [8]:
POSCAR = Poscar(deformed_Si_DC)
print(POSCAR)

Si2
1.0
0.000000 2.460874 2.460874
2.460874 0.000000 2.460874
2.460874 2.460874 0.000000
Si
2
direct
0.500000 0.500000 0.500000 Si
0.750000 0.750000 0.750000 Si



In [9]:
def write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory="new_input_set", use_fake_potcar=True):
    
    import os
    if not os.path.exists(directory):
        os.makedirs(directory)

    if not use_fake_potcar:
        POTCAR.write_file("{}/POTCAR".format(directory))
    else:
        from shutil import copyfile
        # Copy over a fake potcar
        copyfile("fake_vasp_data/3kvi0H85jC/POTCAR", "directory")
    
    POSCAR.write_file("{}/POSCAR".format(directory))
    INCAR.write_file("{}/INCAR".format(directory))
    KPOINTS.write_file("{}/KPOINTS".format(directory))
    print("Input set written to '{}'".format(directory))

In [10]:
my_directory = "my_part_2/Si_DC"
write_input_set(None, POSCAR, INCAR, KPOINTS, directory=my_directory)

Input set written to 'my_part_2/Si_DC'


In [11]:
Poscar.from_file("my_part_2/Si_DC/POSCAR")

Si2
1.0
0.000000 2.460874 2.460874
2.460874 0.000000 2.460874
2.460874 2.460874 0.000000
Si
2
direct
0.500000 0.500000 0.500000 Si
0.750000 0.750000 0.750000 Si

In [12]:
!python fake_vasp.py "my_part_2/Si_DC"


No matching pre-computed data for the input files in directory 'my_part_2/Si_DC' were found:
    Matching POSCAR found? False
    Matching POSCAR + KPOINTS found? Unknown
    Matching POSCAR + KPOINTS + INCAR found? Unknown
Please check your input files and try again.



In [13]:
from pymatgen.io.vasp.outputs import Outcar
OUTCAR = Outcar(my_directory + "/OUTCAR")
OUTCAR.final_energy

-10.84255253